In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import seaborn as sns
import random
from pprint import pprint
from sklearn.datasets import load_iris
%matplotlib inline

In [46]:
iris=load_iris()

In [47]:
df=pd.DataFrame(iris.data,columns=iris.feature_names)

In [48]:
df.columns=["sepal_l","sepal_w","petal_l","petal_w"]

In [49]:
df["label"]=iris.target

In [50]:
# df.label[df.label==0]="setosa"
# df.label[df.label==1]="verginica"
# df.label[df.label==2]="versicolor"
df.label.replace(0,"setosa",inplace=True)
df.label.replace(1,"virginica",inplace=True)
df.label.replace(2,"versicolor",inplace=True)

df

,sepal_l,sepal_w,petal_l,petal_w,label
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,versicolor
146,6.3,2.5,5.0,1.9,versicolor
147,6.5,3.0,5.2,2.0,versicolor
148,6.2,3.4,5.4,2.3,versicolor


In [51]:
def train_test_split(df,test_size):
    indices=df.index.tolist()
    test_indices=random.sample(population=indices,k=test_size)
    test_df=df.loc[test_indices]
    train_df=df.drop(test_indices)
    return train_df,test_df

In [88]:
train_df,test_df=train_test_split(df,40)
test_df.head()

,sepal_l,sepal_w,petal_l,petal_w,label
104,6.5,3.0,5.8,2.2,versicolor
118,7.7,2.6,6.9,2.3,versicolor
145,6.7,3.0,5.2,2.3,versicolor
16,5.4,3.9,1.3,0.4,setosa
2,4.7,3.2,1.3,0.2,setosa


In [107]:
def check_purity(data):
    label_column=data[:,-1]
    unique_classes=np.unique(label_column)
    if len(unique_classes)==1:
        return(True)
    else:
        return(False)

In [95]:
data=train_df.values

In [109]:
check_purity(data)

False

In [102]:
pd.Series(label_column).value_counts()

virginica     40
versicolor    38
setosa        32
dtype: int64